[View in Colaboratory](https://colab.research.google.com/github/ergintuncer/Sezen-aksu-ark-s-yazan-yapay-zeka-diyorum/blob/master/Verisetinin%20olu%C5%9Fturulmas%C4%B1%20(Beautiful%20Soup%20&%20Web%20Scraping).ipynb)

![Sezen aksu şarkıları](https://image.ibb.co/d4BoQf/anasayfa.png)

### Gerekli kütüphanelerin import edilmesi

In [0]:
import requests
import csv
from bs4 import BeautifulSoup, NavigableString, Tag
from google.colab import files
import pandas as pd

### Tüm şarkıların adlarının ve adreslerinin seçilmesi

In [0]:
# Sanatşı sayfasındaki verilerin alınması
sayfa = requests.get('https://www.sarkisozleri.bbs.tr/sanatci/181/sezen-aksu')
soup = BeautifulSoup(sayfa.text, 'html.parser')

# şarkıların bulunduğu div'in seçilmesi
sarkilar = soup.find(class_='col-sm-8 col-md-6 col-lg-8')

# div etiketinin içindeki <a> taglarının seçilmesi
sarki_adlari = sarkilar.find_all('a')
print(sarki_adlari)

![Anasayfa kod seçimi](https://preview.ibb.co/fuoeWL/anasayfa-kod-se-imi.png)

### Sözlerin satır satır kaydedilmesi

In [0]:
# sarki sözlerinin yazılacağı dosyanın oluşturulması ve başlık satırının eklenmesi
f = csv.writer(open('Sezen Aksu.csv', 'w', newline='', encoding='utf-8-sig'))
f.writerow(['Sanatçı Adı','Şarkı Id','Şarkı Adı', 'Şarkı Sözü','Satır No'])
sarki_id=1 # her bir şarkıya ait id

# her bir şarkı için dön
for sarki in sarki_adlari:
    sarki_adi = sarki.contents[0]
    print(sarki_adi)
    sarki_link = "https://www.sarkisozleri.bbs.tr"+sarki.get('href')
    sarki_sayfasi = requests.get(sarki_link)
    soup = BeautifulSoup(sarki_sayfasi.text, 'html.parser') # sözlerin bulunduğu sayfayı parse et
    
    sarki_sozu = soup.findAll(class_='col-md-6') # sadece sözlerin bulunduğu div'i seç
    satir_no=1
    
    # her bir şarkı sözü için dön
    for sarki_satirlar in sarki_sozu:
        for br in sarki_satirlar.findAll('br'): # sözleri satır satır kaydedeceğimiz için her bir <br> tagından böl
            sarki_satir=br.previous_sibling # birinci satıra geri git
            
            # <br> taglarını sil ve sözleri satır satı kaydet
            if not (sarki_satir and isinstance(sarki_satir,NavigableString)):
                continue
            sarki_satir_2 = sarki_satir.nextSibling
            
            if sarki_satir_2 and isinstance(sarki_satir_2,Tag) and sarki_satir_2.name == 'br':
                soz = str(sarki_satir).strip()
                if soz:
                    f.writerow(['Sezen Aksu',sarki_id,str(sarki_adi).strip(), soz,satir_no]) 
                    satir_no=satir_no+1
    sarki_id=sarki_id+1;

![Sarki sözlerinin seçilmesi](https://preview.ibb.co/fKFRBL/sarki-s-zleri-kod.png)



---

Verisetinin önizlemesi

In [0]:
dataset = pd.read_csv('Sezen Aksu.csv', encoding = "utf_8")
dataset.head()

,Sanatçı Adı,Şarkı Id,Şarkı Adı,Şarkı Sözü,Satır No
0,Sezen Aksu,1,1945,Bin Dokuzyüz Kırkbeş,1
1,Sezen Aksu,1,1945,Gel asırlardan uzanda tut ellerimi sımsıcak,2
2,Sezen Aksu,1,1945,Yoksa bendeki çocukda böyle çaresiz kalacak,3
3,Sezen Aksu,1,1945,Öfke ile beslenen çocuklar yalnızdırlar,4
4,Sezen Aksu,1,1945,Ve ümitleri çiçeklerden acıları tarihlerden,5



### Dosyanın bilgisayara indirilmesi

In [0]:
files.download('Sezen Aksu.csv')